# 6 深度学习计算

## 6.1 层和块

单个神经网络：（1）接受一些输入；（2）生成相应的标量输出；（3）具有一组相关参数（parameters）

当考虑具有多个输出的网络时，利用矢量化算法描述整层神经元。（1）接受一组输入；（2）生成相应的输出，（3）由一组可调参数描述。

神经网络块（block）：描述单个层、由多个层组成的组件或整个模型本身。

从编程的角度看，块由类（class）表示。它的任何子类都必须顶一个将输入转换为输出的前向传播函数，并且必须存储任何必须的参数。最后，为了计算梯度，块必须具有反向传播函数。


In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
'''
nn.Sequential定义了一种特殊的Module，表示一个块的类，维护了一个由Module组成的有序列表。
net(X)调用模型获得输出，实际上时net.__call__(X)的缩写。该前向传播函数非常简单，将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入
'''


net = nn.Sequential(
    nn.Linear(20,256),
    nn.ReLU(),
    nn.Linear(256,10)
)

X = torch.rand(2,20)
net(X)

#### 自定义块

每个块必须提供的功能：
1. 将输入数据作为其前向传播的参数
2. 通过前向传播函数来生成输出
3. 计算其输出关于输入的梯度，可通过反向传播函数进行访问
4. 存储和访问前向传播计算所需的参数
5. 根据需要初始化模型参数

编写一个自定义多层感知机，具有256个隐藏单元的隐藏层和一个输出层。MLP类继承了表示块的类。我们只需实现构造函数（__init__函数）和前向传播函数。


In [ ]:
class MLP(nn.Module):
    # 用模型参数声明层。在此声明两个全连接的层
    def __init__(self, input_dim, out_dim):
        # 调用父类Module的构造函数离开执行必要的初始化
        super().__init__()
        self.hidden = nn.Linear(input_dim, 256) # 隐藏层
        self.out = nn.Linear(256, out_dim) # 输出层
    # 定义模型的前向传播，即如何根据输入X返回所需的输出
    def forward(self, X):
        return F.softmax(self.out(F.relu(self.hidden(X))))

net = MLP(20,10)
net(X)

块的一个主要有点就是其多功能性。可以子类化块一创建层（如全连接的类）、整个模型（如上面的MLP）或具有中等复杂度的各种组件。

#### 顺序块

Sequential类徐定义两个关键函数：
1. 将块逐个追加到列表中的函数
2. 前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”

下面的MySequential类提供了与默认Sequential类相同的功能

In [ ]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # module时Module子类的一个实例
            # _module的类型时OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证按照成员添加的顺序便利他们，主要优势时哎模块的参数初始化过程中，系统知道在_modules字典中查找需要初始化参数的子块。

        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256,10), nn.Softmax())
net(X)

#### 在前向传播函数中执行代码

并不是所有的架构都是简单的顺序架构。当需要更强的灵活性时，我们需要自定义块。如在前向传播中执行Python控制流和任意的数学运算。

网络中的所有操作都对网络的激活值及网络的参数起作用。然而，有时希望合并既不是上一层的结果也不是可更新参数的项，称之为常数参数（constant parameter）。如

$$
f(\mathbf{x,w})=c\cdot\mathbf{w^\top x}
$$
其中，$\mathbf x$是参数，$\mathbf w$是参数，c是某个在优化过程中没有更新的指定常量

In [ ]:
class FixedHiddenMLP(nn.Module):
    def __init__(self, input_dim, out_dim):
        super().__init__()
        # 不计算随机权重的梯度。因此其在训练期间保持不变
        self.rand_weight = torch.rand((256, out_dim), requires_grad=False)
        self.linear = nn.Linear(input_dim, 256)
    
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数及relu和mm函数, 常量参数不会被反向传播更新
        X = F.relu(torch.mm(X, self.rand_weight)+1)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP(20,10)
net(X)

In [ ]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256,10), nn.Softmax())
net(X)

## 6.2 参数管理

### 参数访问

从已有模型中访问参数。当通过Sequential类定义模型时，可以通过索引来访问模型的任意层。这就像模型是一个列表一样，每层的参数都在其属性中。

In [ ]:
# 访问第二个全连接层的参数

net[2].state_dict()

### 目标参数

每个参数都表示为参数类的一个实例。对参数执行任何操作都需要访问底层的数值。
参数是复合的对象，包含值，梯度和额外信息。没调用反向传播，所以参数的梯度处于初始状态

In [ ]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)